In [ ]:
!pip install scitime
!pip install pyarabic
!pip install farasapy
!pip install pyarabic
!git clone https://github.com/aub-mind/arabert/
!cd arabert && git checkout 6a58ca118911ef311cbe8cdcdcc1d0360

     |████████████████████████████████| 45.9MB 93kB/s 
     |████████████████████████████████| 112kB 6.3MB/s 
  Created wheel for pyarabic: filename=PyArabic-0.6.10-cp36-none-any.whl size=113324 sha256=192b8c597ba4d8ad97d4b8ba92a51bbc009b3f093b87bc54e0cddef581a5f7df
  Stored in directory: /root/.cache/pip/wheels/10/b8/f5/b7c1a50e6efb83544844f165a9b134afe7292585465e29b61d
Successfully built pyarabic
Cloning into 'arabert'...
remote: Enumerating objects: 200, done.
remote: Counting objects: 100% (200/200), done.
remote: Compressing objects: 100% (147/147), done.
remote: Total 414 (delta 94), reused 148 (delta 47), pack-reused 214
Receiving objects: 100% (414/414), 3.82 MiB | 16.11 MiB/s, done.
Resolving deltas: 100% (217/217), done.
Note: checking out '6a58ca118911ef311cbe8cdcdcc1d0360'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing anot

In [ ]:
!pip install psutil
!pip install joblib

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from scipy.sparse import hstack
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from scitime import Estimator
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.pipeline import FeatureUnion
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report
from farasa.segmenter import FarasaSegmenter
from arabert.preprocess_arabert import preprocess


farasa_segmenter = FarasaSegmenter(interactive=True)

100%|██████████| 241M/241M [00:22<00:00, 11.5MiB/s]

[2021-01-21 23:35:09,601 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

DATA_COLUMN = 'line'
LABEL_COLUMN = 'dialect'
train_files = ["/content/drive/MyDrive/trainset/AE_targe_preprocessed.txt", "/content/drive/MyDrive/trainset/BH_targe_preprocessed.txt","/content/drive/MyDrive/trainset/DZ_targe_preprocessed.txt",
               "/content/drive/MyDrive/trainset/EG_targe_preprocessed.txt", "/content/drive/MyDrive/trainset/IQ_targe_preprocessed.txt","/content/drive/MyDrive/trainset/JO_targe_preprocessed.txt",
               "/content/drive/MyDrive/trainset/KW_targe_preprocessed.txt", "/content/drive/MyDrive/trainset/LB_targe_preprocessed.txt","/content/drive/MyDrive/trainset/LY_targe_preprocessed.txt",
               "/content/drive/MyDrive/trainset/QA_targe_preprocessed.txt", "/content/drive/MyDrive/trainset/OM_targe_preprocessed.txt","/content/drive/MyDrive/trainset/PL_targe_preprocessed.txt",
               "/content/drive/MyDrive/trainset/OM_targe_preprocessed.txt", "/content/drive/MyDrive/trainset/SA_targe_preprocessed.txt","/content/drive/MyDrive/trainset/SD_targe_preprocessed.txt",
               "/content/drive/MyDrive/trainset/SY_targe_preprocessed.txt", "/content/drive/MyDrive/trainset/TN_targe_preprocessed.txt","/content/drive/MyDrive/trainset/YE_targe_preprocessed.txt"]

country_names = ["AE","BH","DZ","EG","IQ","JO","KW","LB","LY","MA","OM","PL","QA","SA","SD","SY","TN","YE"]

label_map = {
    'AE' : 0,
    'BH' : 1,
    'DZ' : 2,
    'EG' : 3,
    'IQ' : 4,
    'JO' : 5,
    'KW' : 6,
    'LB' : 7,
    'LY' : 8,
    'MA' : 9,
    'OM' : 10,
    'PL' : 11,
    'QA' : 12,
    'SA' : 13,
    'SD' : 14,
    'SY' : 15,
    'TN' : 16,
    'YE' : 17
}

print(train_files)
train_data_list = []
for index in range(len(train_files)):
  train_data_list.append(pd.read_csv(train_files[index], lineterminator= '~', header=None, names = ['line'], nrows = 100))
  train_data_list[index][LABEL_COLUMN] = [country_names[index] for i in range(len(train_data_list[index].index))]

train_data = pd.concat(train_data_list)

test_data = pd.read_csv('/content/drive/MyDrive/testset/QADI_test.txt', sep = '\t', header=None, lineterminator='\n')
test_data.columns = [DATA_COLUMN, LABEL_COLUMN]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['/content/drive/MyDrive/trainset/AE_targe_preprocessed.txt', '/content/drive/MyDrive/trainset/BH_targe_preprocessed.txt', '/content/drive/MyDrive/trainset/DZ_targe_preprocessed.txt', '/content/drive/MyDrive/trainset/EG_targe_preprocessed.txt', '/content/drive/MyDrive/trainset/IQ_targe_preprocessed.txt', '/content/drive/MyDrive/trainset/JO_targe_preprocessed.txt', '/content/drive/MyDrive/trainset/KW_targe_preprocessed.txt', '/content/drive/MyDrive/trainset/LB_targe_preprocessed.txt', '/content/drive/MyDrive/trainset/LY_targe_preprocessed.txt', '/content/drive/MyDrive/trainset/QA_targe_preprocessed.txt', '/content/drive/MyDrive/trainset/OM_targe_preprocessed.txt', '/content/drive/MyDrive/trainset/PL_targe_preprocessed.txt', '/content/drive/MyDrive/trainset/OM_targe_preprocessed.txt', '/content/drive/MyDrive/trainset/SA_targe_preprocessed.txt', '/content/drive/

In [ ]:
test_data.drop(test_data.index[test_data[LABEL_COLUMN] == 'MSA'], inplace = True)

train_data[DATA_COLUMN] = train_data[DATA_COLUMN].apply(lambda x: preprocess(x, do_farasa_tokenization=True , farasa=farasa_segmenter, use_farasapy = True))

train_data[LABEL_COLUMN] = train_data[LABEL_COLUMN].apply(lambda x: label_map[x])

test_data[DATA_COLUMN] = test_data[DATA_COLUMN].apply(lambda x: preprocess(x, do_farasa_tokenization=True , farasa=farasa_segmenter, use_farasapy = True))

test_data[LABEL_COLUMN] = test_data[LABEL_COLUMN].apply(lambda x: label_map[x])

100%|██████████| 241M/241M [00:39<00:00, 11.5MiB/s]

In [ ]:
train_df = pd.DataFrame({
    'id':range(len(train_data)),
    'label':train_data[LABEL_COLUMN],
    'text': train_data[DATA_COLUMN].replace(r'\n', ' ', regex=True)
})

dev_df = pd.DataFrame({
    'id':range(len(test_data)),
    'label':test_data[LABEL_COLUMN],
    'text': test_data[DATA_COLUMN].replace(r'\n', ' ', regex=True)
})

In [ ]:
train_text = train_df['text']
test_text = dev_df['text']
all_text = pd.concat([train_text, test_text])
train_text

0     [مستخدم] ياخي ال+ مدرب أختار +ه و+ ال+ مدرب دخ...
1     شو اللي قاعد يجري ف+ ال+ نصر يا أخو +ه . خسر ا...
2     اللي يبحث عن مشكل +ة ال+ وصل راح يحصل +ها ف+ ا...
3     [مستخدم] أنا مش معترض على تغيير عامر . اللي دخ...
4     تراجع مخيف في مستوى ال+ حارس ال+ كبير ماجد ناص...
                            ...                        
95    شعور يهلك ل+ ما تكتم ال+ وجع بين ك+ و+ بين نفس...
96                [مستخدم] و+ نجهل فوق جهل ال+ جاهل +ين
97    [مستخدم] ب+ خير و+ لا في +ه شي جت ل+ +ه سكت +ة...
98                   إذا طاح ال+ وطن كثر +ت سكاكين +ه !
99    [مستخدم] [مستخدم] [مستخدم] [مستخدم] [مستخدم] [...
Name: text, Length: 1800, dtype: object

In [ ]:
# train_word_features = word_vectorizer.transform(train_text.values.astype('U'))
# test_word_features = word_vectorizer.transform(test_text.values.astype('U'))
# train_word_features

In [ ]:
union = FeatureUnion([("word_tfid", TfidfVectorizer(
                                    sublinear_tf=True,
                                    strip_accents='unicode',
                                    analyzer='word',
                                    token_pattern=r'\w{1,}',
                                    ngram_range=(1, 2),
                                    max_features=10000)),
                      ("char_tfid", TfidfVectorizer(
                                    sublinear_tf=True,
                                    strip_accents='unicode',
                                    analyzer='char',
                                    ngram_range=(2, 6),
                                    max_features=50000)),
                      ])

union.fit(all_text.values.astype('U'))
features = union.transform(train_text.values.astype('U'))

In [ ]:
# mlp = MLPClassifier(random_state=1, hidden_layer_sizes= (64,), batch_size = 32, max_iter=5, verbose=True, learning_rate_init=0.0001)
# mlp.fit(features, train_df['label'])

In [ ]:
# model.save('/content/drive/MyDrive/models/n-grams_5_epochs')
# model.evaluate(test_word_features, test_df['label'])
# import pickle
# pkl_filename = "/content/drive/MyDrive/nlp_models/n_grams_20_epochs.sav"
# with open(pkl_filename, 'wb') as file:
#    pickle.dump(mlp, file)

0     [مستخدم] ياخي ال+ مدرب أختار +ه و+ ال+ مدرب دخ...
1     شو اللي قاعد يجري ف+ ال+ نصر يا أخو +ه . خسر ا...
2     اللي يبحث عن مشكل +ة ال+ وصل راح يحصل +ها ف+ ا...
3     [مستخدم] أنا مش معترض على تغيير عامر . اللي دخ...
4     تراجع مخيف في مستوى ال+ حارس ال+ كبير ماجد ناص...
                            ...                        
95    شعور يهلك ل+ ما تكتم ال+ وجع بين ك+ و+ بين نفس...
96                [مستخدم] و+ نجهل فوق جهل ال+ جاهل +ين
97    [مستخدم] ب+ خير و+ لا في +ه شي جت ل+ +ه سكت +ة...
98                   إذا طاح ال+ وطن كثر +ت سكاكين +ه !
99    [مستخدم] [مستخدم] [مستخدم] [مستخدم] [مستخدم] [...
Name: text, Length: 1800, dtype: object

In [ ]:
test_features = union.transform(test_text.values.astype('U'))


In [ ]:
# predicted = mlp.predict(test_features)

In [ ]:
# print(predicted[10:50])
# print(dev_df["label"][10:50])

In [ ]:
# loaded_model = pickle.load(open(pkl_filename, 'rb'))
# predicted = loaded_model.predict(test_features)


In [ ]:
# print(classification_report(dev_df['label'], predicted))
# cf_matrix = confusion_matrix(dev_df['label'], predicted)
# import seaborn as sns
# import matplotlib.pyplot as plt
# df_cm = pd.DataFrame(cf_matrix, index = ["AE","BH","DZ","EG","IQ","JO","KW","LB","LY","MA","OM","PL","QA","SA","SD","SY","TN","YE"],
#                   columns = ["AE","BH","DZ","EG","IQ","JO","KW","LB","LY","MA","OM","PL","QA","SA","SD","SY","TN","YE"])
# plt.figure(figsize = (10,7))
# sns.heatmap(df_cm, annot=True, fmt='g' )

In [ ]:
# predicted = loaded_model.predict_proba(test_features)
# preds_array = np.array(predicted)

In [ ]:
np.savetxt("neural_n_grams.csv", preds_array, delimiter=",")

In [ ]:
# from sklearn.svm import SVC
# svm = SVC(kernel = 'linear', gamma='auto', verbose = True, max_iter = 600)
# svm.fit(features, train_df['label'])

[LibSVM]

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=600, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=True)

In [ ]:
predicted = svm.predict(test_features)
print(classification_report(dev_df['label'], predicted))
cf_matrix = confusion_matrix(dev_df['label'], predicted)
macro_f1 = f1_score(dev_df['label'], predicted,average='macro')
# macro_precision = precision_score(dev_df['label'], predicted,average='macro')
# macro_recall = recall_score(dev_df['label'], predicted,average='macro')
acc = accuracy_score(dev_df['label'], predicted)
print("macro f1: " + str(macro_f1))
print("macro precision: " + str(macro_precision))
print("macro recall: " +str(macro_recall))
print("accuracy: " +str(acc))
import seaborn as sns
import matplotlib.pyplot as plt
df_cm = pd.DataFrame(cf_matrix, index = ["AE","BH","DZ","EG","IQ","JO","KW","LB","LY","MA","OM","PL","QA","SA","SD","SY","TN","YE"],
                  columns = ["AE","BH","DZ","EG","IQ","JO","KW","LB","LY","MA","OM","PL","QA","SA","SD","SY","TN","YE"])
plt.figure(figsize = (10,7))
sns.heatmap(df_cm, annot=True, fmt='g' )

KeyboardInterrupt: ignored